In [1]:
import os
import copy
import tempfile
import json
from subprocess import call
from diConstants import (HG19_ALL_CHROMS, MM9_ALL_CHROMS,
    HG19_TRAIN_CHROMS, MM9_TRAIN_CHROMS,
    VALID_CHROMS, TEST_CHROMS) 

import models
import modelTemplates

Using Theano backend.


In [2]:
GM_MARKS = ['H3K27AC'] # Is this output mark?
#GM_MARKS = ['H3K27AC', 'H3K4ME1', 'H3K4ME3', 'H3K27ME3', 'H3K36ME3']
modelList = []
def init_test_GM18526():
    for test_cell_line in ['GM18526']:
        for subsample_target_string in ['0.5e6']:
            for predict_binary_output in [False, True]: # original : TRUE , FALSE   
                for output_mark in GM_MARKS:                            

                    model_params = modelTemplates.make_model_params(
                                #### Prediction model(keras ver) specification ####
                                model_library='keras',
                                model_class='SeqToSeq',
                                model_type='dcgan',
                                model_specific_params={
                                    'num_filters': 6,
                                    'filter_length': 251
                                },
                                compile_params={            
                                    'optimizer': 'adagrad'
                                },

                                #### Dataset specification ####
                                dataset_params={
                                    'train_dataset_name': 'GM12878_5+1marks-K4me3_all',
                                    'test_dataset_name': '%s_5+1marks-K4me3_all' % test_cell_line, #(1)
                                    'num_train_examples': 1000,
                                    'seq_length': 1001,
                                    'peak_fraction': 0.5,                            
                                    'train_X_subsample_target_string': subsample_target_string, #(2)
                                    'num_bins_to_test': None,
                                    'train_chroms': HG19_ALL_CHROMS, # GM-12878 ch1 ~ ch22
                                    'test_chroms': HG19_ALL_CHROMS, # CM-18526 ch1 ~ ch22
                                    'only_chr1': True
                                },
                                output_marks=[output_mark],
                                train_params={
                                    'nb_epoch': 10,
                                    'batch_size': 100
                                },
                                predict_binary_output=predict_binary_output,
                                zero_out_non_bins=True,
                                generate_bigWig=True)

                    init_model(model_params)

def init_model(model_params):
    m = models.SeqModel.instantiate_model(model_params)
    modelList.append([m,model_params])
    print '\n\n\n'

In [3]:
init_test_GM18526()

Generator model summary
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, 1001, 6)       9042        convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 1001, 6)       0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 1001, 1)       1507        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 1001, 1)       0           convolution1d_2[0][0]            
Total params: 10,549
Trainable params: 10,549
Non-trainable params:

In [4]:
def train_model():
    print 'Number of Model:', len(modelList)
    for model, params in modelList:
        print model
        model.train_model()
    
train_model()

Number of Model: 2
X-train: (1000, 1001, 6)
Y-train: (1000, 1001, 1)
{'compile_params': {'loss': 'MSE', 'optimizer': 'adagrad'}, 'random_seed': 0, 'output_marks': ['H3K27AC'], 'predict_binary_output': False, 'input_marks': [u'H3K27AC', u'H3K27ME3', u'H3K36ME3', u'H3K4ME1', u'H3K4ME3', u'INPUT'], 'filter_length': 251, 'train_params': {'nb_epoch': 10, 'validation_split': 0.2, 'batch_size': 100}, 'scale_input': '01', 'generate_bigWig': True, 'zero_out_non_bins': True, 'model_library': 'keras', 'num_filters': 6, 'model_type': 'dcgan', 'model_class': 'SeqToSeq', 'dataset_params': {'test_datasets': [<dataset.Dataset object at 0x7fb5a1bcea90>], 'train_dataset': <dataset.Dataset object at 0x7fb5a1bcef90>, 'only_chr1': True, 'num_bins_to_test': None, 'seq_length': 1001}}
Epoch is 0
Number of batches 100
(  1 /  10) G loss :   2.007792
(  2 /  10) G loss :   1.897956


KeyboardInterrupt: 

In [ ]:
def evaluate_model():
    for model, params in modelList:
        print model.evaluate_model()

#evaluate_model()